# Problema de Designação de Pessoas para Instrumentos e Vozes em Dias Específicos

## Conjuntos
- **i**: Conjunto de pessoas.
- **j**: Conjunto de instrumentos.
- **k**: Conjunto de dias.

## Parâmetros e Variáveis
- **P(i, j)**: Matriz binária indicando se a pessoa **i** pode tocar o instrumento **j**.
- **L(j)**: Peso de cada instrumento **j** (variando de 0 a 10).
- **D['Peso'][k]**: Peso de cada dia **k** (variando de 0 a 10).
- **Disp(i, k)**: Disponibilidade da pessoa **i** no dia **k**.
- **DPM['Quantidade'][i]**: Quantidade de dias que a pessoa **i** se ofereceu para ajudar por mês.

### Variável de Decisão
- **X(i, j, k)**: Matriz de solução binária indicando se a pessoa **i** toca o instrumento **j** no dia **k**.

## Função Objetivo
A função objetivo maximiza a eficácia da alocação de pessoas para tocar instrumentos levando em consideração a disponibilidade, habilidades e os pesos atribuídos aos dias, instrumentos e vozes. A fórmula matemática é:

Max Somatório em {i,j,k} de X(i,j,k) * Disp(i,k) * P(i,j) * L(j) * D['Peso'][k]

## Restrições
1. **Cada instrumento é tocado por uma pessoa por dia**:
   - Cada instrumento deve ser tocado por uma única pessoa em um dia.
   
      Somatório em {i} de X(i,j,k) <= 1, ∀j, ∀k

2. **Cada pessoa toca somente um instrumento por dia**:
   - Uma pessoa só pode tocar um instrumento em um dia.
   
      Somatório em {j} de X(i,j,k) <= 1, ∀i, ∀k

3. **Cada pessoa toca somente o número de dias que se ofereceu por mês**:
   - A quantidade de dias que uma pessoa se ofereceu para tocar não pode ser excedida.
   
      Somatório em {j} de (X(i,j,k,) + X(i,j,k+1) + X(i,j,k+2) + X(i,j,k+3)) <= DPM['Quantidade'][i] ∀i, ∀k

4. **Pessoa não será alocada se não tocar o instrumento específico**:
   - Se uma pessoa não sabe tocar um instrumento específico, ela não será alocada para tocá-lo.
   
      X(i,j,k) = 0, se P(i,j) = 0, ∀i, ∀j, ∀k

### Descrição do Modelo
   O modelo busca alocar pessoas para tocar instrumentos em determinadas vozes nos dias disponíveis, maximizando a utilização das suas habilidades e disponibilidades. A função objetivo valoriza as alocações que melhor utilizam a disponibilidade das pessoas e os pesos atribuídos aos dias, instrumentos e vozes. As restrições garantem uma alocação eficiente e justa, respeitando as capacidades e disponibilidades de cada pessoa, e assegurando que cada recurso (instrumento, voz) seja utilizado adequadamente.

In [5]:
import pandas as pd
from pulp import *

file_name = 'Dados Louvor Shalom - V1.xlsx'

sheets_dict = pd.read_excel(file_name, sheet_name=['Instrumento', 'Peso dos Instrumentos', 'Peso dos Dias','Disponibilidade', 'Dias por Mês'])

# Acesse cada sheet pelo nome
#Dias por Mês
P = sheets_dict['Instrumento'].set_index('Pessoas')

L = sheets_dict['Peso dos Instrumentos'].set_index('Instrumentos').squeeze()
D = sheets_dict['Peso dos Dias'].set_index('Dias')

Disp = sheets_dict['Disponibilidade'].set_index('Dias')
DPM = sheets_dict['Dias por Mês'].set_index('Pessoas')

# Convertendo os índices para o tipo datetime
D.index = pd.to_datetime(D.index)
Disp.index = pd.to_datetime(Disp.index)

# Formatando os índices para o formato desejado
D.index = D.index.strftime('%d_%m_%Y')
Disp.index = Disp.index.strftime('%d_%m_%Y')

In [6]:
# Definir o problema
prob = LpProblem("Alocacao_de_Musicos", LpMaximize)

# Variáveis de decisão
X = LpVariable.dicts("X", (P.index, P.columns, D.index), cat='Binary')

# Função objetivo
prob += lpSum([X[i][j][k] * Disp[i][k] * P.loc[i, j] * L[j] * D['Peso'][k] for i in P.index for j in P.columns for k in D.index])

# Restrições

# Um instrumento só é tocado por uma pessoa em cada dia K
for j in P.columns: #para todo instrumento
    for k in D.index: #para todo dia
        prob += lpSum([X[i][j][k] for i in P.index]) <= 1 #Cada pessoa 

# Cada pessoa toca somente um instrumento cada dia
for i in P.index: #Cada pessoa 
    for k in D.index: #para todo dia
        prob += lpSum([X[i][j][k] for j in P.columns]) <= 1 #para todo instrumento


# Cada pessoa toca somente o numero de dias que se ofereceu por mês
for i in P.index: 
    for k in range(len(D.index) - 3):
        prob += lpSum([X[i][j][D.index[k]] + X[i][j][D.index[k+1]] + X[i][j][D.index[k+2]] + X[i][j][D.index[k+3]] for j in P.columns]) <= DPM['Quantidade'][i]


# A pessoa não será alocada se não tocar o instrumento específico
for i in P.index: #Pessoas
    for j in P.columns: #Instrumentos
        for k in D.index: #Dias
            if P.loc[i, j] == 0: #Tabela de Capacidades. Pessoa i toca o instrumento j.
                prob += X[i][j][k] == 0

# Resolver o problema
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/henrique/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2ce1a9610ddb444eafe97d9d640f7be6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/2ce1a9610ddb444eafe97d9d640f7be6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 928 COLUMNS
At line 7754 RHS
At line 8678 BOUNDS
At line 9524 ENDATA
Problem MODEL has 923 rows, 845 columns and 4849 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1591 - 0.01 seconds
Cgl0002I 559 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 13 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 13 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 13 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0

1

In [8]:
# Imprimir o status da solução
print("Status:", LpStatus[prob.status])

# Imprimir o valor da função objetivo
print("Valor da função objetivo =", value(prob.objective))

# Dicionário para armazenar os resultados
resultados = {}

# Iterar sobre cada variável de decisão
for v in prob.variables():
    if v.varValue != 0:
        # Extrair o nome da pessoa, o instrumento e o dia
        _, pessoa, instrumento, dia = v.name.split("_", 3)

        # Se o dia ainda não está no dicionário, adicione-o
        if dia not in resultados:
            resultados[dia] = {}

        # Adicione a pessoa e o instrumento ao dia correspondente
        resultados[dia][instrumento] = pessoa

df = pd.DataFrame(columns=P.columns, index=D.index)
escala_instrumental = pd.DataFrame(0, columns=P.index, index=D.index)

# Preencher o DataFrame com os resultados
for dia in resultados:
    for instrumento in resultados[dia]:
        row_index = dia
        column_name = resultados[dia][instrumento]
        escala_instrumental.loc[row_index, column_name] = 1
        df.loc[dia, instrumento] = resultados[dia][instrumento]

# # Substituir NaN por ''
df = df.fillna('')

# Convertendo os índices para o tipo datetime
df.index = pd.to_datetime(df.index, format='%d_%m_%Y')

# Formatando os índices para o formato desejado e criando um novo índice
formatted_index = df.index.strftime('%d/%m/%Y')

# Atribuindo o novo índice formatado ao DataFrame
df.index = formatted_index
df

Status: Optimal
Valor da função objetivo = 1591.0


,Bateria,Baixo,Teclado,Violão,Guitarra
Dias,,,,,
08/06/2024,Israel,Gabriel,Mauri,Ricardo,Osvaldo
15/06/2024,Lucas,Gabriel,Vinicius,Daniel,Enzo
22/06/2024,Henrique,Osvaldo,Mauri,Ricardo,Daniel
29/06/2024,Marcela,Gabriel,Vinicius,Rafael,Enzo
06/07/2024,Israel,Gabriel,Bernardo,Ricardo,Osvaldo
13/07/2024,Henrique,Gabriel,Bernardo,Ricardo,Daniel
20/07/2024,Henrique,Osvaldo,Mauri,Ricardo,Daniel
27/07/2024,Lucas,Gabriel,Vinicius,Rafael,Enzo
03/08/2024,Marcela,Gabriel,Vinicius,Ricardo,Enzo
